Load file in Data Lake

In [0]:
file_path = "dbfs:/FileStore/cardealer/df_car_raw.csv"

df_car_raw = spark.read.option("header", "true") \
    .option("escape", "\"") \
    .option("multiline", "true") \
    .option("quote", "\"") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .csv(file_path)

df_car_raw.show(10)


+----------+--------------------+------------+------------+----------+----------+
|     marca|              modelo|         ano|          km|    cambio|     preco|
+----------+--------------------+------------+------------+----------+----------+
|      FIAT|    MOBI LIKE 1.0 4P|2022/2023\n||60.925 km\n||    MANUAL| R$ 54.290|
|    TOYOTA|COROLLA ALTIS HYB...|2021/2022\n||25.918 km\n||AUTOMÁTICO|R$ 155.190|
|    TOYOTA|COROLLA CROSS XR ...|2021/2022\n||24.954 km\n||AUTOMÁTICO|R$ 124.190|
|   RENAULT|       LOGAN ZEN 1.0|2022/2023\n||48.120 km\n||    MANUAL| R$ 64.390|
|VOLKSWAGEN|          VOYAGE 1.0|2022/2023\n||22.111 km\n||    MANUAL| R$ 62.990|
|      FIAT|UNO ATTRACTIVE EV...|2020/2020\n||95.633 km\n||    MANUAL| R$ 47.990|
|   RENAULT|     KWID ZEN 1.0 4P|2022/2023\n||18.187 km\n||    MANUAL| R$ 57.290|
|VOLKSWAGEN|T-CROSS 200 TSI A...|2022/2022\n||45.430 km\n||AUTOMÁTICO|R$ 103.990|
|VOLKSWAGEN|      GOL MPI 1.0 4P|2022/2023\n||68.410 km\n||    MANUAL| R$ 57.490|
|VOLKSWAGEN|    

**BRONZE LAYER**

In [0]:
bronze_path = "dbfs:/FileStore/cardealer//bronze"
df_car_bronze = df_car_raw.write.format("delta").mode("overwrite").save(bronze_path)

In [0]:
df_car_bronze = spark.read.format("delta").load("dbfs:/FileStore/cardealer/bronze")
df_car_bronze.show(10)

+----------+--------------------+------------+------------+----------+----------+
|     marca|              modelo|         ano|          km|    cambio|     preco|
+----------+--------------------+------------+------------+----------+----------+
|      FIAT|    MOBI LIKE 1.0 4P|2022/2023\n||60.925 km\n||    MANUAL| R$ 54.290|
|    TOYOTA|COROLLA ALTIS HYB...|2021/2022\n||25.918 km\n||AUTOMÁTICO|R$ 155.190|
|    TOYOTA|COROLLA CROSS XR ...|2021/2022\n||24.954 km\n||AUTOMÁTICO|R$ 124.190|
|   RENAULT|       LOGAN ZEN 1.0|2022/2023\n||48.120 km\n||    MANUAL| R$ 64.390|
|VOLKSWAGEN|          VOYAGE 1.0|2022/2023\n||22.111 km\n||    MANUAL| R$ 62.990|
|      FIAT|UNO ATTRACTIVE EV...|2020/2020\n||95.633 km\n||    MANUAL| R$ 47.990|
|   RENAULT|     KWID ZEN 1.0 4P|2022/2023\n||18.187 km\n||    MANUAL| R$ 57.290|
|VOLKSWAGEN|T-CROSS 200 TSI A...|2022/2022\n||45.430 km\n||AUTOMÁTICO|R$ 103.990|
|VOLKSWAGEN|      GOL MPI 1.0 4P|2022/2023\n||68.410 km\n||    MANUAL| R$ 57.490|
|VOLKSWAGEN|    

**SILVER LAYER**

In [0]:
dbutils.fs.rm("dbfs:/FileStore/cardealer/silver", recurse=True)


Out[171]: True

In [0]:
silver_path = "dbfs:/FileStore/cardealer/silver"
df_car_silver = df_car_bronze.write.format("delta").mode("overwrite").save(silver_path)

In [0]:
df_car_silver = spark.read.format("delta").load("dbfs:/FileStore/cardealer/bronze")
df_car_silver.show(10)

+----------+--------------------+------------+------------+----------+----------+
|     marca|              modelo|         ano|          km|    cambio|     preco|
+----------+--------------------+------------+------------+----------+----------+
|      FIAT|    MOBI LIKE 1.0 4P|2022/2023\n||60.925 km\n||    MANUAL| R$ 54.290|
|    TOYOTA|COROLLA ALTIS HYB...|2021/2022\n||25.918 km\n||AUTOMÁTICO|R$ 155.190|
|    TOYOTA|COROLLA CROSS XR ...|2021/2022\n||24.954 km\n||AUTOMÁTICO|R$ 124.190|
|   RENAULT|       LOGAN ZEN 1.0|2022/2023\n||48.120 km\n||    MANUAL| R$ 64.390|
|VOLKSWAGEN|          VOYAGE 1.0|2022/2023\n||22.111 km\n||    MANUAL| R$ 62.990|
|      FIAT|UNO ATTRACTIVE EV...|2020/2020\n||95.633 km\n||    MANUAL| R$ 47.990|
|   RENAULT|     KWID ZEN 1.0 4P|2022/2023\n||18.187 km\n||    MANUAL| R$ 57.290|
|VOLKSWAGEN|T-CROSS 200 TSI A...|2022/2022\n||45.430 km\n||AUTOMÁTICO|R$ 103.990|
|VOLKSWAGEN|      GOL MPI 1.0 4P|2022/2023\n||68.410 km\n||    MANUAL| R$ 57.490|
|VOLKSWAGEN|    

In [0]:
df_car_silver = df_car_silver.withColumnRenamed("marca", "brand") \
    .withColumnRenamed("modelo", "model") \
    .withColumnRenamed("ano", "year") \
    .withColumnRenamed("km", "mileage") \
    .withColumnRenamed("cambio", "gearbox") \
    .withColumnRenamed("preco", "price")
df_car_silver.show(5)

+----------+--------------------+------------+------------+----------+----------+
|     brand|               model|        year|     mileage|   gearbox|     price|
+----------+--------------------+------------+------------+----------+----------+
|      FIAT|    MOBI LIKE 1.0 4P|2022/2023\n||60.925 km\n||    MANUAL| R$ 54.290|
|    TOYOTA|COROLLA ALTIS HYB...|2021/2022\n||25.918 km\n||AUTOMÁTICO|R$ 155.190|
|    TOYOTA|COROLLA CROSS XR ...|2021/2022\n||24.954 km\n||AUTOMÁTICO|R$ 124.190|
|   RENAULT|       LOGAN ZEN 1.0|2022/2023\n||48.120 km\n||    MANUAL| R$ 64.390|
|VOLKSWAGEN|          VOYAGE 1.0|2022/2023\n||22.111 km\n||    MANUAL| R$ 62.990|
+----------+--------------------+------------+------------+----------+----------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import regexp_replace

for col_name in df_car_silver.columns:
    df_car_silver = df_car_silver.withColumn(col_name, regexp_replace(df_car_silver[col_name], "\n", ""))

df_car_silver.show(truncate=False)


+----------+------------------------------+----------+----------+----------+----------+
|brand     |model                         |year      |mileage   |gearbox   |price     |
+----------+------------------------------+----------+----------+----------+----------+
|FIAT      |MOBI LIKE 1.0 4P              |2022/2023||60.925 km||MANUAL    |R$ 54.290 |
|TOYOTA    |COROLLA ALTIS HYBRID AT 1.8 4P|2021/2022||25.918 km||AUTOMÁTICO|R$ 155.190|
|TOYOTA    |COROLLA CROSS XR FLEX 2.0 4P  |2021/2022||24.954 km||AUTOMÁTICO|R$ 124.190|
|RENAULT   |LOGAN ZEN 1.0                 |2022/2023||48.120 km||MANUAL    |R$ 64.390 |
|VOLKSWAGEN|VOYAGE 1.0                    |2022/2023||22.111 km||MANUAL    |R$ 62.990 |
|FIAT      |UNO ATTRACTIVE EVO 1.0 4P     |2020/2020||95.633 km||MANUAL    |R$ 47.990 |
|RENAULT   |KWID ZEN 1.0 4P               |2022/2023||18.187 km||MANUAL    |R$ 57.290 |
|VOLKSWAGEN|T-CROSS 200 TSI AT 1.0 4P     |2022/2022||45.430 km||AUTOMÁTICO|R$ 103.990|
|VOLKSWAGEN|GOL MPI 1.0 4P      

In [0]:
from pyspark.sql.functions import regexp_replace
df_car_silver = df_car_silver \
    .withColumn("year", regexp_replace("year", "\\|", "")) \
    .withColumn("mileage", regexp_replace("mileage", " km|\\|", "")) \
    .withColumn("price", regexp_replace("price", "R\\$", ""))

df_car_silver.show(truncate=False)



+----------+------------------------------+---------+-------+----------+--------+
|brand     |model                         |year     |mileage|gearbox   |price   |
+----------+------------------------------+---------+-------+----------+--------+
|FIAT      |MOBI LIKE 1.0 4P              |2022/2023|60.925 |MANUAL    | 54.290 |
|TOYOTA    |COROLLA ALTIS HYBRID AT 1.8 4P|2021/2022|25.918 |AUTOMÁTICO| 155.190|
|TOYOTA    |COROLLA CROSS XR FLEX 2.0 4P  |2021/2022|24.954 |AUTOMÁTICO| 124.190|
|RENAULT   |LOGAN ZEN 1.0                 |2022/2023|48.120 |MANUAL    | 64.390 |
|VOLKSWAGEN|VOYAGE 1.0                    |2022/2023|22.111 |MANUAL    | 62.990 |
|FIAT      |UNO ATTRACTIVE EVO 1.0 4P     |2020/2020|95.633 |MANUAL    | 47.990 |
|RENAULT   |KWID ZEN 1.0 4P               |2022/2023|18.187 |MANUAL    | 57.290 |
|VOLKSWAGEN|T-CROSS 200 TSI AT 1.0 4P     |2022/2022|45.430 |AUTOMÁTICO| 103.990|
|VOLKSWAGEN|GOL MPI 1.0 4P                |2022/2023|68.410 |MANUAL    | 57.490 |
|VOLKSWAGEN|GOL 

In [0]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, DecimalType

df_car_silver = df_car_silver \
    .withColumn("year", regexp_extract(col("year"), r'(\d{4})', 1).cast(IntegerType())) \
    .withColumn("mileage", regexp_replace(col("mileage"), r'\.', '').cast(IntegerType())) \
    .withColumn("price", col("price").cast(DecimalType(10, 2)))

df_car_silver.show(truncate=False)

+----------+------------------------------+----+-------+----------+------+
|brand     |model                         |year|mileage|gearbox   |price |
+----------+------------------------------+----+-------+----------+------+
|FIAT      |MOBI LIKE 1.0 4P              |2022|60925  |MANUAL    |54.29 |
|TOYOTA    |COROLLA ALTIS HYBRID AT 1.8 4P|2021|25918  |AUTOMÁTICO|155.19|
|TOYOTA    |COROLLA CROSS XR FLEX 2.0 4P  |2021|24954  |AUTOMÁTICO|124.19|
|RENAULT   |LOGAN ZEN 1.0                 |2022|48120  |MANUAL    |64.39 |
|VOLKSWAGEN|VOYAGE 1.0                    |2022|22111  |MANUAL    |62.99 |
|FIAT      |UNO ATTRACTIVE EVO 1.0 4P     |2020|95633  |MANUAL    |47.99 |
|RENAULT   |KWID ZEN 1.0 4P               |2022|18187  |MANUAL    |57.29 |
|VOLKSWAGEN|T-CROSS 200 TSI AT 1.0 4P     |2022|45430  |AUTOMÁTICO|103.99|
|VOLKSWAGEN|GOL MPI 1.0 4P                |2022|68410  |MANUAL    |57.49 |
|VOLKSWAGEN|GOL MPI 1.0 4P                |2022|47916  |MANUAL    |59.29 |
|FIAT      |ARGO FLEX 1.0

In [0]:
from pyspark.sql.functions import year, current_date, regexp_extract, col

current_year = year(current_date())
df_car_silver = df_car_silver.withColumn("age", current_year - col("year"))
df_car_silver.show(truncate=False)


+----------+------------------------------+----+-------+----------+------+---+
|brand     |model                         |year|mileage|gearbox   |price |age|
+----------+------------------------------+----+-------+----------+------+---+
|FIAT      |MOBI LIKE 1.0 4P              |2022|60925  |MANUAL    |54.29 |2  |
|TOYOTA    |COROLLA ALTIS HYBRID AT 1.8 4P|2021|25918  |AUTOMÁTICO|155.19|3  |
|TOYOTA    |COROLLA CROSS XR FLEX 2.0 4P  |2021|24954  |AUTOMÁTICO|124.19|3  |
|RENAULT   |LOGAN ZEN 1.0                 |2022|48120  |MANUAL    |64.39 |2  |
|VOLKSWAGEN|VOYAGE 1.0                    |2022|22111  |MANUAL    |62.99 |2  |
|FIAT      |UNO ATTRACTIVE EVO 1.0 4P     |2020|95633  |MANUAL    |47.99 |4  |
|RENAULT   |KWID ZEN 1.0 4P               |2022|18187  |MANUAL    |57.29 |2  |
|VOLKSWAGEN|T-CROSS 200 TSI AT 1.0 4P     |2022|45430  |AUTOMÁTICO|103.99|2  |
|VOLKSWAGEN|GOL MPI 1.0 4P                |2022|68410  |MANUAL    |57.49 |2  |
|VOLKSWAGEN|GOL MPI 1.0 4P                |2022|4791

In [0]:
silver_path = "dbfs:/FileStore/cardealer/silver"
df_car_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .save(silver_path)
df_car_silver.show(5)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|      FIAT|    MOBI LIKE 1.0 4P|2022|  60925|    MANUAL| 54.29|  2|
|    TOYOTA|COROLLA ALTIS HYB...|2021|  25918|AUTOMÁTICO|155.19|  3|
|    TOYOTA|COROLLA CROSS XR ...|2021|  24954|AUTOMÁTICO|124.19|  3|
|   RENAULT|       LOGAN ZEN 1.0|2022|  48120|    MANUAL| 64.39|  2|
|VOLKSWAGEN|          VOYAGE 1.0|2022|  22111|    MANUAL| 62.99|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 5 rows



In [0]:
df_car_silver = spark.read.format("delta").load("dbfs:/FileStore/cardealer/silver")
df_car_silver.show(10)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|      FIAT|    MOBI LIKE 1.0 4P|2022|  60925|    MANUAL| 54.29|  2|
|    TOYOTA|COROLLA ALTIS HYB...|2021|  25918|AUTOMÁTICO|155.19|  3|
|    TOYOTA|COROLLA CROSS XR ...|2021|  24954|AUTOMÁTICO|124.19|  3|
|   RENAULT|       LOGAN ZEN 1.0|2022|  48120|    MANUAL| 64.39|  2|
|VOLKSWAGEN|          VOYAGE 1.0|2022|  22111|    MANUAL| 62.99|  2|
|      FIAT|UNO ATTRACTIVE EV...|2020|  95633|    MANUAL| 47.99|  4|
|   RENAULT|     KWID ZEN 1.0 4P|2022|  18187|    MANUAL| 57.29|  2|
|VOLKSWAGEN|T-CROSS 200 TSI A...|2022|  45430|AUTOMÁTICO|103.99|  2|
|VOLKSWAGEN|      GOL MPI 1.0 4P|2022|  68410|    MANUAL| 57.49|  2|
|VOLKSWAGEN|      GOL MPI 1.0 4P|2022|  47916|    MANUAL| 59.29|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 10 rows



In [0]:
df_car_silver.select('brand').show(5)

+----------+
|     brand|
+----------+
|      FIAT|
|    TOYOTA|
|    TOYOTA|
|   RENAULT|
|VOLKSWAGEN|
+----------+
only showing top 5 rows



In [0]:
df_car_silver.select('brand', 'price').show(3)

+------+------+
| brand| price|
+------+------+
|  FIAT| 54.29|
|TOYOTA|155.19|
|TOYOTA|124.19|
+------+------+
only showing top 3 rows



In [0]:
df_car_silver.filter(col('price')< 100.000).show(3)

+----------+----------------+----+-------+-------+-----+---+
|     brand|           model|year|mileage|gearbox|price|age|
+----------+----------------+----+-------+-------+-----+---+
|      FIAT|MOBI LIKE 1.0 4P|2022|  60925| MANUAL|54.29|  2|
|   RENAULT|   LOGAN ZEN 1.0|2022|  48120| MANUAL|64.39|  2|
|VOLKSWAGEN|      VOYAGE 1.0|2022|  22111| MANUAL|62.99|  2|
+----------+----------------+----+-------+-------+-----+---+
only showing top 3 rows



In [0]:
df_car_silver.where('price < 150.000').show(4)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|      FIAT|    MOBI LIKE 1.0 4P|2022|  60925|    MANUAL| 54.29|  2|
|    TOYOTA|COROLLA CROSS XR ...|2021|  24954|AUTOMÁTICO|124.19|  3|
|   RENAULT|       LOGAN ZEN 1.0|2022|  48120|    MANUAL| 64.39|  2|
|VOLKSWAGEN|          VOYAGE 1.0|2022|  22111|    MANUAL| 62.99|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 4 rows



In [0]:
df_car_silver.where((col('price') > 100.000) & (col('price') < 150.000)).show(4)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|    TOYOTA|COROLLA CROSS XR ...|2021|  24954|AUTOMÁTICO|124.19|  3|
|VOLKSWAGEN|T-CROSS 200 TSI A...|2022|  45430|AUTOMÁTICO|103.99|  2|
|      FIAT|TORO ENDURANCE FL...|2021|  38426|AUTOMÁTICO|104.49|  3|
|VOLKSWAGEN|T-CROSS HIGHLINE 1.4|2020|  42395|AUTOMÁTICO|119.49|  4|
+----------+--------------------+----+-------+----------+------+---+
only showing top 4 rows



In [0]:
df_car_silver.where(col("price") < 70.000).where(col("brand") != "FIAT").show(2)

+----------+-------------+----+-------+-------+-----+---+
|     brand|        model|year|mileage|gearbox|price|age|
+----------+-------------+----+-------+-------+-----+---+
|   RENAULT|LOGAN ZEN 1.0|2022|  48120| MANUAL|64.39|  2|
|VOLKSWAGEN|   VOYAGE 1.0|2022|  22111| MANUAL|62.99|  2|
+----------+-------------+----+-------+-------+-----+---+
only showing top 2 rows



In [0]:
df_car_silver.select("brand", "mileage").distinct().count()

Out[198]: 172

In [0]:

df_car_silver.sort("mileage").show(5)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|VOLKSWAGEN|NIVUS HIGHLINE 20...|2022|   3896|AUTOMÁTICO|133.99|  2|
|     CHERY|  TIGGO 8 TXS 7L 1.6|2022|   9433|AUTOMÁTICO|179.59|  2|
|      FIAT|    MOBI LIKE 1.0 4P|2022|  13411|    MANUAL| 59.79|  2|
| CHEVROLET|        ONIX LTZ 1.0|2023|  15520|    MANUAL| 90.49|  1|
|      FIAT|    MOBI LIKE 1.0 4P|2022|  15611|    MANUAL| 60.29|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 5 rows



In [0]:

df_car_silver.orderBy("mileage", "price").show(5)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|VOLKSWAGEN|NIVUS HIGHLINE 20...|2022|   3896|AUTOMÁTICO|133.99|  2|
|     CHERY|  TIGGO 8 TXS 7L 1.6|2022|   9433|AUTOMÁTICO|179.59|  2|
|      FIAT|    MOBI LIKE 1.0 4P|2022|  13411|    MANUAL| 59.79|  2|
| CHEVROLET|        ONIX LTZ 1.0|2023|  15520|    MANUAL| 90.49|  1|
|      FIAT|    MOBI LIKE 1.0 4P|2022|  15611|    MANUAL| 60.29|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 5 rows



In [0]:
df_car_silver.orderBy(col("mileage"), col("price")).show(5)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|VOLKSWAGEN|NIVUS HIGHLINE 20...|2022|   3896|AUTOMÁTICO|133.99|  2|
|     CHERY|  TIGGO 8 TXS 7L 1.6|2022|   9433|AUTOMÁTICO|179.59|  2|
|      FIAT|    MOBI LIKE 1.0 4P|2022|  13411|    MANUAL| 59.79|  2|
| CHEVROLET|        ONIX LTZ 1.0|2023|  15520|    MANUAL| 90.49|  1|
|      FIAT|    MOBI LIKE 1.0 4P|2022|  15611|    MANUAL| 60.29|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import expr

df_car_silver.orderBy(expr("price desc")).show(5)


+----------+--------------------+----+-------+-------+-----+---+
|     brand|               model|year|mileage|gearbox|price|age|
+----------+--------------------+----+-------+-------+-----+---+
|      FIAT|UNO ATTRACTIVE EV...|2020|  95633| MANUAL|47.99|  4|
|      FIAT|UNO ATTRACTIVE EV...|2020|  53547| MANUAL|50.99|  4|
|VOLKSWAGEN|      GOL MPI 1.0 4P|2020|  81826| MANUAL|52.29|  4|
|   RENAULT|        KWID ZEN 1.0|2021|  43443| MANUAL|52.99|  3|
|      FIAT|       MOBI LIKE 1.0|2022|  75411| MANUAL|52.99|  2|
+----------+--------------------+----+-------+-------+-----+---+
only showing top 5 rows



In [0]:
df_car_silver.orderBy(col("price").desc(), col("mileage").asc()).show(5)

+----------+--------------------+----+-------+----------+------+---+
|     brand|               model|year|mileage|   gearbox| price|age|
+----------+--------------------+----+-------+----------+------+---+
|     VOLVO|XC40 T4 MOMENTUM 2.0|2020|  34307|AUTOMÁTICO|205.39|  4|
|     CHERY|  TIGGO 8 TXS 7L 1.6|2022|   9433|AUTOMÁTICO|179.59|  2|
|    TOYOTA|COROLLA ALTIS HYB...|2021|  25918|AUTOMÁTICO|155.19|  3|
|MITSUBISHI|ECLIPSE CROSS HPE...|2022|  47270|AUTOMÁTICO|146.99|  2|
|     CHERY|    TIGGO 5X PRO 1.5|2022|  16866|AUTOMÁTICO|146.09|  2|
+----------+--------------------+----+-------+----------+------+---+
only showing top 5 rows



In [0]:
unique_brands = df_car_silver.select("brand").distinct()
unique_brands.show()

+----------+
|     brand|
+----------+
|   HYUNDAI|
|      FIAT|
|    TOYOTA|
|   CITROEN|
|      FORD|
|   PEUGEOT|
| CHEVROLET|
|      JEEP|
|     VOLVO|
|VOLKSWAGEN|
|     CHERY|
|   RENAULT|
|MITSUBISHI|
+----------+



In [0]:
df_car_silver.filter("brand = 'FIAT'").show(5)


+-----+--------------------+----+-------+----------+-----+---+
|brand|               model|year|mileage|   gearbox|price|age|
+-----+--------------------+----+-------+----------+-----+---+
| FIAT|    MOBI LIKE 1.0 4P|2022|  60925|    MANUAL|54.29|  2|
| FIAT|UNO ATTRACTIVE EV...|2020|  95633|    MANUAL|47.99|  4|
| FIAT|    ARGO FLEX 1.0 4P|2022|  65625|AUTOMÁTICO|61.90|  2|
| FIAT|    MOBI LIKE 1.0 4P|2022|  13411|    MANUAL|59.79|  2|
| FIAT|    MOBI LIKE 1.0 4P|2022|  15611|    MANUAL|60.29|  2|
+-----+--------------------+----+-------+----------+-----+---+
only showing top 5 rows



In [0]:
count_volkswagen = df_car_silver.filter("brand = 'VOLKSWAGEN'").count()
print(f' There are {count_volkswagen} of Wolkswagen brand')

 There are 35 of Wolkswagen brand


**GOLD LAYER**

In [0]:
gold_path = "dbfs:/FileStore/cardealer/gold"
df_car_gold = df_car_silver.write.format("delta").mode("overwrite").save(gold_path)